### Introduction
This notebook loads events from a database and lo file to produce a CSV file to be imported into IBM Process Mining.

### Setup
Import the required python libraries and configre the database connection.

In [1]:
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as psql
from sqlalchemy import create_engine

Settings and credentials for postgres database

In [2]:
user = ''
port = 0
db = ''
host = ''
pwd = ''

In [32]:
engine = create_engine(f"postgresql://{user}:{pwd}@{host}:{port}/{db}")

### Load Data
We'll load everything to keep things simple but in reality you'd probbaly want to select from a specific period.

In [4]:
order_sql = "select * from orders;"
orders = pd.read_sql_query(order_sql, engine)

In [5]:
invoice_sql = "select * from invoices;"
invoices = pd.read_sql_query(invoice_sql, engine)

In [30]:
customer_sql = "select * from customers;"
customers = pd.read_sql_query(customer_sql, engine)

We'll preview the data so we know what we're working with.

In [33]:
orders.head()

,order_id,created_date,customer_id,installation_date,status,created_by,installation_scheduled,install_id,installed_by
0,101,2021-08-05,C1001,2021-08-28,COMP,Dave,2021-08-14,I221,Suzy
1,102,2021-08-06,C1002,2021-08-29,COMP,Jim,2021-08-16,I257,Suzy
2,103,2021-08-07,C1003,2021-08-30,COMP,Fred,2021-08-17,I307,Jake


In [8]:
customers.head()

,customer_id,Name,Phone,email,addr1,addr2,postcode
0,C1001,Fred Smith,07111 123456,fred@goof.com,29 High Street,Trumpton,TT12 1TV
1,C1002,Jason Bourne,07111 123456,jason@yah.com,8 High Street,Trumpton,TT12 1TV


In [9]:
invoices.head(5)

,invoice_id,order_id,invoice_date,amount,payment_date,created_by
0,21-1001A,101,2021-08-06,823.45,2021-08-12,Sally
1,21-1002A,102,2021-08-07,745.45,2021-08-09,Sally
2,21-1003A,103,2021-08-12,1245.45,2021-08-21,Sally
3,21-1001B,101,2021-08-30,112.00,2021-09-01,Sally
4,21-1002B,102,2021-08-30,237.71,2021-09-02,Sally


# Event Retrieval
Find the variou events stored in the data.

### Event : Order Created
Find the records in the orders table that correspond to order creation and record these events in the event dataframe. This is the template for all event retrieval cells. 

1. Create an empty list for the events
2. Iterate over the orders
3. Retrieve the data representing the event
4. Create a python dictionary to represent the event
5. Add the new event to the list
6. Once all orders have been processed convert the list of event dictionaries into a panda dataframe.

In some cases this won't be the optimal method (more selective SQL or SQL joins could be faster).

In [34]:
# activity is used to store the event name, the value here will be the activiy name in process mining 
activity = 'Order Created'

# an empty list to store order events
order_created_list = []

# iterate over all orders in the dataframe
for i, order in orders.iterrows():
    # extract timestamp, user and order_id (order)id will be the process id) 
    timestamp = order['created_date']
    user = order['created_by']
    order_id = order['order_id']
    
    # Create a python dictionary to hold the data form the event
    event = {'order_id': order_id, 'timestamp': timestamp, 'activity': activity, 'user': user}
    # Add the event tp our list
    order_created_list.append(event)

# Use a pandas utility to convert a list of dictionaries into a dataframe    
order_created_events = pd.DataFrame(order_created_list)

Preview our first events:

In [35]:
event

{'order_id': 103,
 'timestamp': Timestamp('2021-08-07 00:00:00'),
 'activity': 'Order Created',
 'user': 'Fred'}

In [11]:
order_created_events.head()

,order_id,timestamp,activity,user
0,101,2021-08-05,Order Created,Dave
1,102,2021-08-06,Order Created,Jim
2,103,2021-08-07,Order Created,Fred


### Event : Invoices Raised
Invoice date of the invoices associated with the order. This means iterating over every order and for each order doing a search of the invoice data. For each order there will be three invoices but we'll just use the date from the first invoice as they should all be the same.

This is the same approach as before except it features a join between the order data and the invoice data. pandas provides a way for us to write a query against the invoice dataframe using the order number from the order. The f"" notation is just a formatted string that substitutes the value of the order_id variable. 

In [53]:
# activity is used to store the event name, the value here will be the activiy name in process mining 
activity = 'Invoice Raised'
invoice_raised_list = []

#iterate over the orders
for i, order in orders.iterrows():
    
    order_id = order['order_id']
    
    #for each order_id find the matching invoices in the invoice data
    query_str = f"order_id == {order_id}"
    matching_invoices = invoices.query(query_str)
    
    #If the order has no invoice just skip to the next order
    if len(matching_invoices.index) == 0:
        continue
    
    #Get the date for the first matching index
    # iloc is index location, so we're taking the timestamp 
    # and user from the invoice at position 0 
    timestamp = matching_invoices.iloc[0]['invoice_date']
    user = matching_invoices.iloc[0]['created_by']
    
    #Create the event
    event = {'order_id': order_id, 'timestamp': timestamp, 'activity': activity, 'user': user}
    invoice_raised_list.append(event)
    
invoice_raised_events = pd.DataFrame(invoice_raised_list)

In [55]:
query_str

'order_id == 103'

In [56]:
matching_invoices

,invoice_id,order_id,invoice_date,amount,payment_date,created_by
2,21-1003A,103,2021-08-12,1245.45,2021-08-21,Sally
5,21-1003B,103,2021-08-30,125.44,2021-09-03,Sally
8,21-1003C,103,2021-08-30,350.00,2021-09-08,Sally


In [57]:
matching_invoices['amount'].sum()

1720.89

In [54]:
invoice_raised_events.head()

,order_id,timestamp,activity,user
0,101,2021-08-06,Invoice Raised,Sally
1,102,2021-08-07,Invoice Raised,Sally
2,103,2021-08-12,Invoice Raised,Sally


### Event : Register Deposit Payment
Look for invoice with A suffix to have a payment date

In [42]:
# activity is used to store the event name, the value here will be the activiy name in process mining 
activity = 'Deposit Paid'
deposit_paid_list = []

#iterate over the orders
for i, order in orders.iterrows():
    
    order_id = order['order_id']
    
    #for each order_id find the matching invoices in the invoice data
    query_str = f"order_id == {order_id}"
    matching_invoices = invoices.query(query_str)
    
    if len(matching_invoices.index) == 0:
        continue
    
    #Does the invoice number end with A
    for i, inv in matching_invoices.iterrows():
        invoice_id = inv['invoice_id']
        payment_date = inv['payment_date']
        
        if invoice_id.endswith('A') and payment_date != "":
            timestamp = inv['payment_date']
            event = {'order_id': order_id,  'timestamp': timestamp, 'activity': activity, 'user': 'robot', 'invoice_id': invoice_id,}
            deposit_paid_list.append(event)
    
deposit_paid_events = pd.DataFrame(deposit_paid_list)

In [43]:
deposit_paid_events.head()

,order_id,timestamp,activity,user,invoice_id
0,101,2021-08-12,Deposit Paid,robot,21-1001A
1,102,2021-08-09,Deposit Paid,robot,21-1002A
2,103,2021-08-21,Deposit Paid,robot,21-1003A


### Event : Schedule Job
Installation scheduled date on order

In [16]:
# activity is used to store the event name, the value here will be the activiy name in process mining 
activity = 'Scheduled'
scheduled_list = []
for i, order in orders.iterrows():
    timestamp = order['installation_scheduled']
    user = order['created_by']
    order_id = order['order_id']
    
    if timestamp != "":
    
        event = {'order_id': order_id, 'timestamp': timestamp, 'activity': activity, 'user': user}
        scheduled_list.append(event)
    
scheduled_events = pd.DataFrame(scheduled_list)

In [17]:
scheduled_events

,order_id,timestamp,activity,user
0,101,2021-08-14,Scheduled,Dave
1,102,2021-08-16,Scheduled,Jim
2,103,2021-08-17,Scheduled,Fred


### Installation
Look for orders where installation_date is set

In [18]:
activity = 'Installation'
installation_list = []
for i, order in orders.iterrows():
    timestamp = order['installation_date']
    user = order['installed_by']
    order_id = order['order_id']
    install_id = order['install_id']
    
    if timestamp != "" and install_id:
    
        event = {'order_id': order_id, 'install_id': install_id, 'timestamp': timestamp, 'activity': activity, 'user': user}
        installation_list.append(event)
    
installation_events = pd.DataFrame(installation_list)

In [19]:
installation_events

,order_id,install_id,timestamp,activity,user
0,101,I221,2021-08-28,Installation,Suzy
1,102,I257,2021-08-29,Installation,Suzy
2,103,I307,2021-08-30,Installation,Jake


### Event : Register Second Payment
Look for invoice with B suffix to have a payment date

In [20]:
activity = 'Register Second Payment'
second_payment_list = []

#iterate over the orders
for i, order in orders.iterrows():
    
    order_id = order['order_id']
    
    #for each order_id find the matching invoices in the invoice data
    query_str = f"order_id == {order_id}"
    matching_invoices = invoices.query(query_str)
    
    if len(matching_invoices.index) == 0:
        continue
    
    #Does the invoice number end with B
    for i, inv in matching_invoices.iterrows():
        invoice_id = inv['invoice_id']
        payment_date = inv['payment_date']
        
        if invoice_id.endswith('B') and payment_date != "":
            timestamp = inv['payment_date']
            event = {'order_id': order_id, 'invoice_id': invoice_id, 'timestamp': timestamp, 'activity': activity, 'user': 'robot'}
            second_payment_list.append(event)
    
second_payment_events = pd.DataFrame(second_payment_list)

In [21]:
second_payment_events

,order_id,invoice_id,timestamp,activity,user
0,101,21-1001B,2021-09-01,Register Second Payment,robot
1,102,21-1002B,2021-09-02,Register Second Payment,robot
2,103,21-1003B,2021-09-03,Register Second Payment,robot


### Event : Register Grant Payment
Look for invoice with C suffix to have a payment date

In [44]:
activity = 'Register Grant Payment'
grant_payment_list = []

#iterate over the orders
for i, order in orders.iterrows():
    
    order_id = order['order_id']
    
    #for each order_id find the matching invoices in the invoice data
    query_str = f"order_id == {order_id}"
    matching_invoices = invoices.query(query_str)
    
    if len(matching_invoices.index) == 0:
        continue
    
    #Does the invoice number end with C
    for i, inv in matching_invoices.iterrows():
        invoice_id = inv['invoice_id']
        payment_date = inv['payment_date']
        
        if invoice_id.endswith('C') and payment_date != "":
            timestamp = inv['payment_date']
            event = {'order_id': order_id, 'invoice_id': invoice_id, 'timestamp': timestamp, 'activity': activity, 'user': 'robot'}
            grant_payment_list.append(event)
    
grant_payment_events = pd.DataFrame(grant_payment_list)

In [45]:
grant_payment_events

,order_id,invoice_id,timestamp,activity,user
0,101,21-1001C,2021-09-08,Register Grant Payment,robot
1,102,21-1002C,2021-09-08,Register Grant Payment,robot
2,103,21-1003C,2021-09-08,Register Grant Payment,robot


### Event : Service Call
Check logfile from ticketing system for any service calls relating to the installation work. This event is quite different to the others as we're not scanning a pandas dataframe, instead we are going line by line through a log file.



In [24]:
# Import the python regualar expression library
import re

tickets_file = open('tickets.log', 'r')

# This is the regular expression that will be used to scan a line of the log file.
# The regex uses named groups to store each component it finds.
pattern = r"(?i)(Ticket Completed:\s+)(?P<ticket>\S+)(\sOrder:\s+)(?P<order>\S+)(\sReported:\s+)(?P<reported>\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})(\sStarted:\s+)(?P<started>\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})(\sFinished:\s+)(?P<finished>\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})(\s+Opcode:\s+)(?P<op_code>\S+)(\s+Cost:\s+)(?P<cost>\S+)"
activity = 'Service Call'
service_list = []

for log_entry in tickets_file.readlines():
    match = re.search(pattern, log_entry)
    
    if match:
        md = match.groupdict()
        order_id = md['order']
        ticket_id = md['ticket']
        timestamp = md['reported']
        completed = md['finished']
        user = md['op_code']
        cost = md['cost']
        
        event = {'order_id': order_id, 
                 'timestamp': timestamp, 
                 'activity': activity, 
                 'user': user, 
                 'ticket_id': ticket_id, 
                 'completed': completed,
                 'cost': cost}
        service_list.append(event)
        
tickets_file.close()
service_events = pd.DataFrame(service_list)

In [25]:
service_events

,order_id,timestamp,activity,user,ticket_id,completed,cost
0,101,2021-08-29 13:45:33,Service Call,122,T12112,2021-09-01 10:25:00,22.32
1,101,2021-09-02 09:22:21,Service Call,122,T12113,2021-09-03 14:45:00,49.99


### Merge the dataframes
The various event dataframes we've created can now be combined. Pandas will perform the concatenation, taking care of the differnt columns automatically. 

In [26]:
events = pd.concat(objs = [order_created_events,
                           invoice_raised_events,
                           deposit_paid_events, 
                           scheduled_events,
                           installation_events, 
                           second_payment_events, 
                           grant_payment_events, 
                           service_events], ignore_index=True)


Lets see what we produced:

In [27]:
events

,order_id,timestamp,activity,user,invoice_id,install_id,ticket_id,completed,cost
0,101,2021-08-05 00:00:00,Order Created,Dave,NaN,NaN,NaN,NaN,NaN
1,102,2021-08-06 00:00:00,Order Created,Jim,NaN,NaN,NaN,NaN,NaN
2,103,2021-08-07 00:00:00,Order Created,Fred,NaN,NaN,NaN,NaN,NaN
3,101,2021-08-06 00:00:00,Invoice Raised,Sally,NaN,NaN,NaN,NaN,NaN
4,102,2021-08-07 00:00:00,Invoice Raised,Sally,NaN,NaN,NaN,NaN,NaN
5,103,2021-08-12 00:00:00,Invoice Raised,Sally,NaN,NaN,NaN,NaN,NaN
6,101,2021-08-12 00:00:00,Deposit Paid,robot,21-1001A,NaN,NaN,NaN,NaN
7,102,2021-08-09 00:00:00,Deposit Paid,robot,21-1002A,NaN,NaN,NaN,NaN
8,103,2021-08-21 00:00:00,Deposit Paid,robot,21-1003A,NaN,NaN,NaN,NaN
9,101,2021-08-14 00:00:00,Scheduled,Dave,NaN,NaN,NaN,NaN,NaN


This is ok, but to improve the readability we'll re-order the columns.

In [28]:
# Reorder the dataframe 
events = events[['order_id', 'invoice_id', 'install_id','timestamp', 'activity', 'user', 'ticket_id', 'completed', 'cost']]
events

,order_id,invoice_id,install_id,timestamp,activity,user,ticket_id,completed,cost
0,101,NaN,NaN,2021-08-05 00:00:00,Order Created,Dave,NaN,NaN,NaN
1,102,NaN,NaN,2021-08-06 00:00:00,Order Created,Jim,NaN,NaN,NaN
2,103,NaN,NaN,2021-08-07 00:00:00,Order Created,Fred,NaN,NaN,NaN
3,101,NaN,NaN,2021-08-06 00:00:00,Invoice Raised,Sally,NaN,NaN,NaN
4,102,NaN,NaN,2021-08-07 00:00:00,Invoice Raised,Sally,NaN,NaN,NaN
5,103,NaN,NaN,2021-08-12 00:00:00,Invoice Raised,Sally,NaN,NaN,NaN
6,101,21-1001A,NaN,2021-08-12 00:00:00,Deposit Paid,robot,NaN,NaN,NaN
7,102,21-1002A,NaN,2021-08-09 00:00:00,Deposit Paid,robot,NaN,NaN,NaN
8,103,21-1003A,NaN,2021-08-21 00:00:00,Deposit Paid,robot,NaN,NaN,NaN
9,101,NaN,NaN,2021-08-14 00:00:00,Scheduled,Dave,NaN,NaN,NaN


### Export to CSV
Now we can export the event dataframe to a CSV.

In [29]:
events.to_csv('events.csv', date_format='%d-%m-%Y %H:%M:%S')